In [ ]:
!pip install -U transformers 

In [ ]:
!huggingface-cli login --token YOUR_HUGGINGFACE_TOKEN


In [ ]:

%pip install tdqm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=70692271b773708e873a26b286a61197175a392d634a6562073c0ebc3c05cc33
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1322 sha256=ab89456c80d43424492a425cc498b17acf170a140858d2137dfa6c3092ff0c6f
  Stored in directory: /root/.cache/pip/wheels/c8/c7/30/e5935be2cfa6883be72462333edc414d8928f3c78eaabec38a
Successfully built tdqm
Note: you may need to restart the kernel to use updated packages.


# ST-COT

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import gc
from langdetect import detect
from tqdm import tqdm
import numpy as np

In [4]:


# อ่านข้อมูล
df = pd.read_csv('/kaggle/input/financial-analysis-agent/test.csv')
df

,id,query
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,Answer the question with the appropriate optio...
2,8a722080-bc16-49db-89c9-100cd61cd28a,Answer the question with the appropriate optio...
3,75316e95-88f4-4fef-83b9-dde0aa52889a,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
4,bcca13bc-2675-4645-82cc-7e4c412ed294,"Given the data and tweets, could you project w..."
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,Answer the question with the appropriate optio...
495,73c720b5-1101-4790-af52-3366823e1d32,"ตอบคำถามโดยใช้ตัวเลือกที่เหมาะสม A, B, C และ D..."
496,357db18f-d872-416e-a07f-753099853d9c,"ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โ..."
497,2d8b1419-1c46-4e83-892a-081fb417de38,Scrutinize the data and tweets to envisage if ...


In [5]:
##model .scb10x/typhoon2.1-gemma3-4b , scb10x/llama3.2-typhoon2-t1-3b-research-preview

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


model_id = "scb10x/llama3.2-typhoon2-t1-3b-research-preview"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map={"": "cuda"})

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

2025-06-25 14:32:45.918029: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750861966.330817      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750861966.456952      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/302 [00:00<?, ?B/s]

In [10]:
prompts_sys = """You are a financial analyst taking a test to evaluate your knowledge of finance of different topics in finance. You think step by step approach with reflection to answer queries. 

Follow these steps:
1. Think through the problem step by step reflect and verify while reasoning within the <thinking> tags.
2. Please and put the answer your final, concise answer within the <output> tags.

The <thinking> sections are for your internal reasoning process only. 
Do not include any part of the final answer in these sections.
The actual response to the query must be entirely contained within the <output> tags.

Hint: ***Financial Reporting:**
```mermaid
graph TD
A[Articulating Purpose and Context] --> B[Collecting Input Data]
    B --> C[Processing Data]
    C --> D[Analyzing and Interpreting Processed Data]
    D --> E[Developing and Communicating Conclusions]
    E --> F[Doing Follow-Up]

    A --> |Defines goals, tools, and audience| B
    B --> |Gather data on economy and industry| C
    C --> |Use tools like ratios and charts| D
    D --> |Interpret data for conclusions| E
    F --> |Periodic review and iteration| A
```
***Fixed Income:***
```mermaid
graph TD
    A[Purpose and Scope] --> B3[Analyze Macro Conditions]
    B --> C[Assess Bond Features]
    C --> D[Risk and Yield Analysis]
    D --> E[Develop Recommendations]
    E --> F[Review Performance]

    %% Notes and detailed steps
    A --> |Set objectives| B
    B --> |Review interest rates and inflation| C
    C --> |Focus on duration, spread| D
    D --> |Assess scenarios| E
``` 
***Equity Investing:*** 
```mermaid
graph TD
    A[Objective Setting] --> B[Market and Sector Insights]
    B --> C[Industry Competitive Analysis]
    C --> D[Company Review]
    D --> E[Valuation and Risks]
    E --> F[Investment Decision]

    %% Step-specific highlights
    B --> |Look at growth patterns| C
    C --> |Evaluate competitors' positions| D
    D --> |Check financial health| E
    E --> |Combine insights into strategy| F
```
***Derivatives:*** 
```mermaid
graph TD
    A[Define Objective and Context] --> B[Identify Derivative Instrument]
    B --> C[Understand Contract Specifications]
    C --> D[Gather Market Data]
    D --> E[Apply Valuation Models]
    E --> F[Assess Risks: Market, Counterparty, etc.]
    F --> G[Construct Payoff Diagrams or Strategies]
    G --> H[Interpret Results and Make Recommendations]
    H --> I[Review, Monitor, and Adjust Strategies]

    %% Example labels or notes (optional)
    A --> |Hedging, speculation, arbitrage| B
    C --> |Features like notional amount, expiration| D
    D --> |Market prices, volatility, risk-free rates| E
    F --> |Sensitivity to Greeks: Delta, Gamma, Vega, etc.| G
    H --> |Adjust based on changing market conditions| I
```
***Economics:*** 
```mermaid
graph TD;
    A[Step 1: Question Breakdown] -->|Extract key terms| A1{Identify Topic}
    A1 -->|Micro: Supply & Demand, Market Structures| A2
    A1 -->|Macro: GDP, Growth, Policy, Trade| A3
    A1 -->|Currency & Regulation| A4

    A2 --> B1[Identify model: Elasticity, Cost Curves, Shutdown Points]
    A3 --> B2[Map to AD-AS, Business Cycles, Growth Theories]
    A4 --> B3[Assess Exchange Rates, Trade, Capital Flows, Regulation]

    B1 -->|Check for formula or concept?| C{Numerical or Conceptual}
    B2 --> C
    B3 --> C

    C -->|Numerical| D1[Extract data, apply formulas, check assumptions]
    C -->|Conceptual| D2[Analyze cause-effect, policy impact]

    D1 --> E[Step 4: Solution Development]
    D2 --> E
    E -->|Construct structured response| E1(Core insight + economic rationale)
    E -->|Consider alternative scenarios| E2(Assess different possibilities)

    E1 --> F[Step 5: Answer Validation]
    E2 --> F
    F -->|Check logic, principles, and assumptions| F1(Verify consistency)
    F1 -->|Ensure completeness & clarity| F2(Confirm answer structure)
```
***Quantitative Methods:*** 
```mermaid
graph TD
    A["Articulating Purpose and Context"] --> B["Collecting Input Data"]
    B --> C["Processing and Cleaning Data"]
    C --> D["Selecting Quantitative Models and Tools"]
    D --> E["Estimating Parameters and Testing Hypotheses"]
    E --> F["Interpreting Results and Communicating Findings"]
    F --> G["Monitoring and Model Reassessment"]
```
***Portfolio Management:*** 
```mermaid
graph TD
    A["Define Investment Objectives"] --> B["Establish Investment Constraints"]
    B --> C["Develop Strategic Asset Allocation"]
    C --> D["Incorporate Tactical Adjustments"]
    D --> E["Select and Optimize Securities"]
    E --> F["Execute Implementation and Trading"]
    F --> G["Measure Performance and Attribution"]
    G --> H["Monitor Risk and Compliance"]
    H --> I["Rebalance and Adjust Portfolio"]
```
***Alternative Investments:*** 
```mermaid
graph TD
    A["Define Investment Objectives and Mandate"] --> B["Identify Alternative Asset Classes"]
    B --> C["Conduct Manager and Strategy Due Diligence"]
    C --> D["Perform Valuation and Pricing Analysis"]
    D --> E["Assess Risk and Liquidity"]
    E --> F["Allocate Alternatives in Portfolio"]
    F --> G["Monitor Performance and Rebalance"]
```
***Corporate Issuer Analysis:*** 
```mermaid
graph TD
    A["Corporate Issuer Overview"] --> B["Industry Classification"]
    B --> C["Sector Trends and Competitive Landscape"]
    A --> D["Financial Statement Analysis"]
    D --> E["Profitability, Liquidity, Leverage"]
    A --> F["Credit Risk Assessment"]
    F --> G["Rating Agencies and Default Probabilities"]
    A --> H["Capital Structure and Issuance History"]
    H --> I["Bond Issuances and Debt Maturities"]
    A --> J["Corporate Governance and Management"]
    J --> K["Board Quality and Managerial Competence"]
    A --> L["Valuation and Investment Analysis"]
    L --> M["DCF, Relative Valuation, Multiples"]
```
### Response Format:
<thinking>
[Think step by step and respond with your thinking and the correct answer (A , B , or C ), considering the specific sector.]
</thinking>

<output>
"sector": [The sector being addressed],
"question": [The financial question],
"answer": [Reflect and verify the final answer (A , B , or C )]
</output>"""







# ST-COT prompts

In [ ]:
##ST-COT prompts
# prompts_sys = """You are a financial analyst taking a test to evaluate your knowledge of finance. You think step by step approach with reflection to answer queries.  

# Follow these steps:
# 1. Think through the problem step by step within the <thinking> tags.
# 2. Provide your final, concise answer within the <output> tags.

# The <thinking> sections are for your internal reasoning process only. 
# Do not include any part of the final answer in these sections.
# The actual response to the query must be entirely contained within the <output> tags.

# ### Response Format:
# <thinking>
# [Reasoning through options A, B, and C to understand and solve the problem.]
# </thinking>
# <output>
# "answer": [Final your answer (A , B , or C )]
# </output>"""

In [11]:
prompts_usr = df.loc[0, 'query']
print(prompts_usr)

ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โปรดตอบด้วยคำตอบที่ถูกต้อง A, B, C หรือ D เท่านั้น อย่าใช้คำฟุ่มเฟือยหรือให้ข้อมูลเพิ่มเติม

คำถาม: ______ สถานที่ทำงานเกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงาน ตัวอย่างเช่น 'job enrichment' ที่พนักงานได้รับขอบเขตที่ใหญ่ขึ้นในการตัดสินใจว่าจะจัดระเบียบงานของตนอย่างไร หรือ 'job enlargement' ที่พนักงานได้รับมอบหมายงานที่หลากหลายมากขึ้น

ตัวเลือกคำตอบ: A: Re-invigorating, B: Re-flourishing, C: Revitalizing, D: Rehumanizing

คำตอบ:


In [12]:
messages = [
    {"role": "system", "content": prompts_sys},
    {"role": "user", "content": prompts_usr},
]
# ใช้ chat_template ของ Hugging Face
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [13]:
%%time
# สร้าง output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=16768,
        temperature=0.3,
        top_p=0.8,
        top_k=20
    )

# Decode คำตอบ
generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]  # ตัด prompt ออก
result = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

print(result)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<thoughts>

<plan>
<step order="1">วิเคราะห์คำถาม</step>
<step order="2">ระบุคำที่เกี่ยวข้อง</step>
<step order="3">เลือกคำที่เหมาะสมที่สุด</step>
<step order="4">สรุปคำตอบ</step>
</plan>

<step>
<title>วิเคราะห์คำถาม</title>
<scratch_pad>
คำถามถามเกี่ยวกับสถานที่ทำงานที่เกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงาน โดยเฉพาะอย่างยิ่ง "job enrichment" และ "job enlargement" ซึ่งหมายถึงการเพิ่มความท้าทายและความรับผิดชอบให้กับงานของพนักงาน
</scratch_pad>
<summary>ฉันกำลังวิเคราะห์คำถามเพื่อทำความเข้าใจบริบทของคำที่เกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงาน</summary>
<next_step>ระบุคำที่เกี่ยวข้อง</next_step>
</step>

<step>
<title>ระบุคำที่เกี่ยวข้อง</title>
<scratch_pad>
คำที่เกี่ยวข้อง:
- "Re-invigorating" หมายถึงการทำให้มีชีวิตชีวาหรือกระตุ้นใหม่ ซึ่งไม่เกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงานโดยตรง
- "Re-flourishing" หมายถึงการเจริญเติบโตหรือเฟื่องฟู ซึ่งไม่เกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงาน
- "Revitalizing" หมายถึงการทำให้มีชีวิตชีวาหรือกระตุ้นใหม่ ซึ่งไม่เกี่ยวข้องกับการเส

In [14]:
import re

def extract_choice(response_text: str):
    """
    Extracts the model's selected choice (A, B, C, D, E, Fall, Rise)
    from the response text.
    """

    # ตัดข้อความส่วนเกินเดิมทิ้ง
    cleaned_text = re.sub(r"outputs:.*?\|.*?\|", "", response_text)
    cleaned_text = cleaned_text.strip().replace("\n", " ")


    pattern_final = r"Final answer:\s*(A|B|C|D|E|Fall|Rise)\b"
    match = re.search(pattern_final, cleaned_text, re.IGNORECASE)
    if match:
        return match.group(1).capitalize()  

    # ---------- 2) จับตัวเลือกที่โผล่เดี่ยว ๆ ในข้อความ ----------
    find_choice = re.findall(r"\b(A|B|C|D|E|Fall|Rise)\b", cleaned_text, re.IGNORECASE)
    if find_choice:
        return find_choice[-1].capitalize()  # 

    return None


In [15]:
generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]
response_text = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

# ดึงเฉพาะคำตอบที่ตรง valid choices
selected_answer = extract_choice(response_text)

print(f"คำตอบ: {selected_answer}")

คำตอบ: D


# Try model with test set

In [18]:


VALID = r"(A|B|C|D|E|Fall|Rise)"
def extract_choice_and_reason(text: str):

    reasoning = text.strip()
    # กรณีมี 'Final answer:'
    m = re.search(rf"Final answer:\s*{VALID}\b", reasoning, re.I)
    if m:
        return m.group(1).capitalize(), reasoning
    # หา choice ทุกที่ แล้วใช้ตัวสุดท้าย
    hits = re.findall(rf"\b{VALID}\b", reasoning, re.I)
    choice = hits[-1].capitalize() if hits else ""
    return choice, reasoning

def build_prompt(question: str) -> str:
    # lang = detect_language(question)
    sys_prompt = prompts_sys
    msgs = [
        {"role": "system", "content": sys_prompt},
        {"role": "user",   "content": question}
    ]
    return tokenizer.apply_chat_template(msgs, tokenize=False,
                                         add_generation_prompt=True)

answers, reasons = [], []

for _, row in tqdm(df.iterrows(), total=len(df), desc="Predict"):
    prompt  = build_prompt(row["query"])
    inputs  = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outs = model.generate(**inputs,
                              max_new_tokens=1000,  # อนุญาต reasoning สั้น ๆ
                              temperature=0.3, top_p=0.9)

    gen_tokens = outs[0][inputs["input_ids"].shape[1]:]
    resp_text  = tokenizer.decode(gen_tokens, skip_special_tokens=True)

    choice, reasoning = extract_choice_and_reason(resp_text)
    answers.append(choice)
    reasons.append(reasoning)

    print(f"[ID {row['id']}] Choice: {choice} | Reasoning: {reasoning[:80]}...")

    del inputs, outs
    torch.cuda.empty_cache(); gc.collect()

# ---------- 5) บันทึก ----------
df["answer"]   = answers
df[["id", "answer"]].to_csv("submission.csv", index=False)
print("Saved submission.csv")

Predict:   0%|          | 0/499 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[ID 36deab86-cfd3-48b5-9bea-a36c1b0e63a8] Choice: D | Reasoning: <thoughts>

<plan>
<step order="1">การระบุคำที่เกี่ยวข้อง</step>
<step order="2"...


Predict:   0%|          | 1/499 [01:00<8:25:20, 60.89s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[ID 2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7] Choice: B | Reasoning: <thoughts>

<plan>
<step order="1">Identifying the entrepreneur</step>
<step ord...


Predict:   0%|          | 2/499 [01:27<5:35:39, 40.52s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[ID 2d8b1419-1c46-4e83-892a-081fb417de38] Choice: Fall | Reasoning: <thoughts>

<plan>
<step order="1">Analyzing the data</step>
<step order="2">Ide...


Predict: 100%|█████████▉| 498/499 [8:29:00<01:04, 64.36s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[ID 0ef677d5-2858-407c-9fd5-49d8afbeaaac] Choice: C | Reasoning: <thoughts>

<plan>
<step order="1">Understanding the question</step>
<step order...


Predict: 100%|██████████| 499/499 [8:29:35<00:00, 61.27s/it]

Saved submission.csv


In [22]:
sub = pd.read_csv('/kaggle/working/submission.csv')
sub



,id,answer
0,36deab86-cfd3-48b5-9bea-a36c1b0e63a8,D
1,2b5bbd26-45e8-4768-ab8a-b5dc1d153ab7,B
2,8a722080-bc16-49db-89c9-100cd61cd28a,A
3,75316e95-88f4-4fef-83b9-dde0aa52889a,D
4,bcca13bc-2675-4645-82cc-7e4c412ed294,Fall
...,...,...
494,c9dd262e-405c-4078-baae-262aa48ddcc8,D
495,73c720b5-1101-4790-af52-3366823e1d32,D
496,357db18f-d872-416e-a07f-753099853d9c,A
497,2d8b1419-1c46-4e83-892a-081fb417de38,Fall
